# controversit: a tool to identify potentially controversial submissions on reddit

This version is for exploration and comes in Jupyter notebook format for ease of use.

The tool will be also implemented in script version and as a web app.

In [1]:
import praw
import numpy as np
import pandas as pd
import warnings; warnings.simplefilter('ignore')

# Setup reddit API

In [2]:
uname = "DavidWithAnE"
upassword = "BROCCO.billy85"
uagent = "controversit by /u/"+uname
cl_id = "l_eEWQRL2Qrvxw"
cl_secret = "QBAlJLF4KPd4ox5ykGTsAoM9Z_A"

reddit = praw.Reddit(client_id=cl_id,
                     client_secret=cl_secret,
                     password=upassword,
                     user_agent=uagent,
                     username=uname)

print("Logged in as user: ",reddit.user.me())

Logged in as user:  DavidWithAnE


# Choose a subreddit and get a list of submissions

In [40]:
def create_pd_dframe(nrows,list_of_fields):
    '''
    A simple function to create an empty
    dataframe with a given number of rows 
    and a given list of columns
    '''
    
    shape = (nrows,len(list_of_fields))
    d = np.empty(shape)
    df = pd.DataFrame(data=d,columns=list_of_fields)
    
    return df

def submission_data_to_dframe(sorting_scheme,n_subm=100):
    '''
    A function that gets n_subm reddit submissions, 
    retrieves their properties, comments to the 
    submission, then saves the results into a pandas 
    dataframe.
    '''

    # Initialize pandas dataframe
    list_of_fields = ["subm_ID","subm_title","subm_author", \
                      "subm_created_utc","subm_upvote_ratio", \
                      "subm_link_flair_text", \
                      "subm_comment_ids", \
                      "subm_comment_authors", \
                      "subm_comment_bodies", \
                      "subm_comment_scores"]
    df = create_pd_dframe(n_subm,list_of_fields)
    df["subm_comment_ids"] = [[]] * len(df)
    df["subm_comment_authors"] = [[]] * len(df)
    df["subm_comment_bodies"] = [[]] * len(df)
    df["subm_comment_scores"] = [[]] * len(df)
        
    # Use reddit api to retrieve the necessary information.
    i = 0 
    for sub in sorting_scheme(limit=n_subm):

        # Data on submissions
        df["subm_ID"].iloc[i] = sub.id
        df["subm_title"].iloc[i] = sub.title
        df["subm_author"].iloc[i] = sub.author
        df["subm_created_utc"].iloc[i] = float(sub.created_utc)
        df["subm_upvote_ratio"].iloc[i] = float(sub.upvote_ratio)
        df["subm_link_flair_text"].iloc[i] = sub.link_flair_text
        
        # Data on comments
        sub.comments.replace_more(limit=0)
        comment_ids = []
        comment_authors = []
        comment_bodies = []
        comment_scores = []
        for com in sub.comments:
            comment_ids.append(com.id)
            comment_authors.append(com.author)
            comment_bodies.append(com.body)
            comment_scores.append(float(com.score))
        df["subm_comment_ids"].iloc[i] = comment_ids
        df["subm_comment_authors"].iloc[i] = comment_authors
        df["subm_comment_bodies"].iloc[i] = comment_bodies
        df["subm_comment_scores"].iloc[i] = comment_scores
        
        # Increase counter
        i=i+1
    
    return df

In [42]:
# Choose a subreddit and a number of submissions
subreddit_name = "politicaldiscussion"
n_subm = 100

subreddit_obj = reddit.subreddit(subreddit_name)

subm_data = submission_data_to_dframe(subreddit_obj.new,n_subm=n_subm)

display(subm_data)

,subm_ID,subm_title,subm_author,subm_created_utc,subm_upvote_ratio,subm_link_flair_text,subm_comment_ids,subm_comment_authors,subm_comment_bodies,subm_comment_scores
0,d3gteo,Democratic Primary Debate,Precursor2552,1.568333e+09,0.88,NaN,"[f02kv2i, f02lxcq, f02oef7, f02o1mh, f02ms2l, ...","[AutoModerator, imrightandyoutknowit, probably...",[[A reminder for everyone](https://www.reddit....,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,d3gius,If a progressive such as Sanders or Warren wer...,FannyPackMasterRace,1.568332e+09,0.83,US Politics,"[f02ih5s, f02kyil, f02l3nv, f02m5gl, f02me4z, ...","[AutoModerator, lvysaur, derstherower, Gemello...",[[A reminder for everyone](https://www.reddit....,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,d31w26,The Necessity of Re-Secreting Congress,bigredfloof,1.568258e+09,0.83,Political Theory,"[ezy625m, ezy6uul, ezy6smn, ezy6hg8, ezy6sk4, ...","[AutoModerator, lacktoesandtolerant, youjerkfa...",[[A reminder for everyone](https://www.reddit....,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,d2wk43,Prime Minister Justin Trudeau officially asked...,adrzz-110,1.568234e+09,0.87,Non-US Politics,"[ezx822q, ezy6iv4, f00bz44, f00lvmi, f01d314]","[AutoModerator, Tired8281, nonpartisanpartisan...",[[A reminder for everyone](https://www.reddit....,"[1.0, 1.0, 1.0, 1.0, 1.0]"
4,d2w5mo,Concerning war-time labor issues between Japan...,FlameZero777,1.568233e+09,0.81,Political History,"[ezx5koq, ezyctm5, ezyz1u6]","[AutoModerator, Bodoblock, semaphore-1842]",[[A reminder for everyone](https://www.reddit....,"[1.0, 1.0, 1.0]"
5,d2rup7,How do the political stances of Obama and Bide...,Levictual,1.568215e+09,0.87,US Elections,"[ezwfhek, ezylpn7, f01w1vi, f005kyu, ezy89ji]","[AutoModerator, small_loan_of_1M, expiredveget...",[[A reminder for everyone](https://www.reddit....,"[1.0, 1.0, 1.0, 1.0, 1.0]"
6,d1rs9m,Should Administrations meet with geopolitical ...,purple_chicken,1.568040e+09,0.92,International Politics,"[ezpg7eh, ezpq9u0, eztbwlp, ezq7vtj, ezqoq1v, ...","[AutoModerator, Bamont, MyBonesToTheSea, small...",[[A reminder for everyone](https://www.reddit....,"[1.0, 198.0, 29.0, 37.0, 31.0, 82.0, 23.0, 4.0..."
7,d1h39x,Several state Republican parties are planning ...,Anxa,1.567977e+09,0.96,US Elections,"[ezlj9m7, ezlpfhp, ezlm5kd, ezlmsj7, ezlotnv, ...","[AutoModerator, Craig1250, notjawn, Leftover-C...",[[A reminder for everyone](https://www.reddit....,"[1.0, 95.0, 331.0, 149.0, 174.0, 103.0, 17.0, ..."
8,d0bovt,Should indigenous communities be given full au...,e12mv2,1.567743e+09,0.98,Legislation,"[ez8gy55, ezkkzos, ezi3b4s, ezjs864, ezjdgqv, ...","[AutoModerator, small_loan_of_1M, WallTheWhite...",[[A reminder for everyone](https://www.reddit....,"[1.0, 49.0, 29.0, 25.0, 9.0, 3.0, 7.0, 1.0, 1...."
9,d09tba,Where do Green Parties stand on the spectrum i...,Cuddlyaxe,1.567733e+09,0.91,International Politics,"[ez82r0n, ezi0x5x, ezjhmhu, ezhksi2, ezkkgvm, ...","[AutoModerator, Firstclass30, Krakors, thereal...",[[A reminder for everyone](https://www.reddit....,"[1.0, 11.0, 31.0, 9.0, 8.0, 2.0, 1.0, 1.0]"


In [84]:
import time
import datetime 

# Save the data
timestamp = time.time()
dati = datetime.datetime.fromtimestamp(timestamp)
da = str(dati)[0:10]
ti = str(dati)[11:19]

fname = "dump_r-"+subreddit_name+"_"+da+"_"+ti+".pkl"
print(fname)
subm_data.to_pickle(fname)

dump_r-politicaldiscussion_2019-09-12_18:02:39.pkl
